In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
df = pd.read_csv("diffusion_db_unaltered.csv")

In [4]:
df.head()

,prompt,image_nsfw,prompt_nsfw
0,"a portrait of a female robot made from code, v...",0.554853,0.001621
1,a portrait of a female robot made from a cloud...,0.153645,0.000707
2,"only memories remain, trending on artstation",0.062496,0.000425
3,dream swimming pool with nobody,0.030799,0.003586
4,a dog doing weights. epic oil painting.,0.181035,0.030822


In [10]:
explicit_keywords = [ 
    "ass", "bitch", "cock", "damn",
    "dick", "fuck", "nude",
    "pussy", "vagina", "nsfw", 
    "slut", "tit", "whore","cum", 
    "dildo", "knocker",
    "boob", "bollock", "schmuck", "choad", "dumbfuck", 
    "jerk", "numbnuts", "seductive",
    "fuckboy", "butthole", 
    "fanny", "munt", "butt", "porn"
]

In [11]:
pattern = r"\b(" + "|".join([f"{word}" for word in explicit_keywords]) + r")\b"
regex = re.compile(pattern, flags=re.IGNORECASE)

In [12]:
sampled_data = pd.read_csv('diffusion_db_unaltered.csv', nrows=2000000)
random_sample = sampled_data.sample(n=2000000, random_state=1)  # Randomly sample 200,000 rows

chunk_size = 2000000  
output_list = []

for start in range(0, len(random_sample), chunk_size):
    chunk = random_sample.iloc[start:start + chunk_size]
    
    # Use .loc to set the 'contains_keywords' column safely
    chunk.loc[:, 'contains_keywords'] = chunk['prompt'].str.contains(pattern, case=False, regex=True, na=False)
    
    # Filter rows that contain the keywords
    keyword_rows = chunk[chunk['contains_keywords']]
    output_list.append(keyword_rows)
    
    # Memory cleanup (optional)
    del chunk
    del keyword_rows

if output_list:
    final_df = pd.concat(output_list, ignore_index=True)
else:
    print("No prompts containing the keywords found.")

C:\Users\mmooreii\AppData\Local\Temp\ipykernel_19744\1945727955.py:11: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  chunk.loc[:, 'contains_keywords'] = chunk['prompt'].str.contains(pattern, case=False, regex=True, na=False)
C:\Users\mmooreii\AppData\Local\Temp\ipykernel_19744\1945727955.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk.loc[:, 'contains_keywords'] = chunk['prompt'].str.contains(pattern, case=False, regex=True, na=False)


In [13]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6169 entries, 0 to 6168
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   prompt             6169 non-null   object 
 1   image_nsfw         6169 non-null   float64
 2   prompt_nsfw        6169 non-null   float64
 3   contains_keywords  6169 non-null   bool   
dtypes: bool(1), float64(2), object(1)
memory usage: 150.7+ KB


In [14]:
tets = final_df
print(tets[["prompt", "contains_keywords"]])

                                                 prompt  contains_keywords
0     close up portrait on the enchanting femenine w...               True
1     majestic gracious regal seductive deity aphrod...               True
2     scarlett johanson doing pole dancing, purple l...               True
3     glamorous and sexy nurse in transparent blouse...               True
4     grungy woman, rainbow hair, soft eyes and narr...               True
...                                                 ...                ...
6164  lee jin - eun as starfire emerging from pink w...               True
6165  glamorous and sexy japanese lawer in chemisier...               True
6166  pop surrealism, lowbrow art, realistic seducti...               True
6167  jellyfish as xenomorph queen, psycho stupid fu...               True
6168  portrait of a red haired woman, in 7 0 s style...               True

[6169 rows x 2 columns]


In [18]:
final_df['prompt'][3213]

'masterpiece beautiful seductive flowing curves preraphaelite face portrait amongst leaves, extreme close up shot, straight bangs, thick set features, yellow ochre ornate medieval dress, horns, amongst foliage mushroom forest arch branching framed with plants, gold gilded circle halo, kilian eng and frederic leighton and rosetti, 4 k '